In [2]:
import torch
import transformers
import json
import os

from transformers import GenerationConfig, LlamaForCausalLM, LlamaTokenizer
from sklearn.metrics import roc_auc_score

import random
import numpy as np

from evaluate import load
bertscore = load("bertscore")

import gc

In [3]:
test_output_path = "./lora_llama2_chat/sample128_valsample3000_epoch50_stratified_eval_loss/test_outputs.json"
with open(test_output_path, 'r') as f:
    test_outputs = json.load(f)
print(f"test outputs: {len(test_outputs)}")

references = [test_output['output'] for test_output in test_outputs]
predictions = [test_output['generated_output'] for test_output in test_outputs]
print(f"references: {references[0]} \n\n Predictions: {predictions[0]}")

test outputs: 3000
references: 
Summary of user behavior and preferences:

The user enjoys heartwarming, charming, and humorous films, often with a romantic or family-oriented theme. They appreciate witty dialogue, engaging storylines, and delightful characters. Animated films and movies that explore personal growth, transformation, and unique perspectives on historical events are also favored. The user tends to avoid darker, more serious themes and prefers to steer clear of sci-fi and crime genres.

Explanation for why the user will like Madeline:

Madeline, a charming and heartwarming family film, aligns perfectly with the user's preferences. Based on the beloved children's book series, it offers adventure, friendship, and a delightful performance by Hatty Jones. The user enjoys stories about personal growth and transformation, and Madeline fits this criteria as it showcases the main character's journey through various challenges. Furthermore, the film's focus on friendship and famil

In [4]:
base_model = "meta-llama/Llama-2-7b-chat-hf"
tokenizer = LlamaTokenizer.from_pretrained(base_model,
                                            padding_side = "left",
                                            add_eos_token = True,
                                            add_bos_token = True,
                                            token = os.environ.get("HUGGINGFACE_ACCESS_TOKEN"),
                                        )
tokenizer.add_special_tokens({"pad_token":tokenizer.eos_token})

max_memory_mapping = {2: "23GiB"}
model = LlamaForCausalLM.from_pretrained(
            base_model,
            load_in_8bit=True,
            torch_dtype=torch.float16,
            device_map="auto",
            max_memory = max_memory_mapping,
            token = os.environ.get("HUGGINGFACE_ACCESS_TOKEN"),
        ).eval()

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [5]:
texts = [references[0], predictions[0]]
t_inputs = tokenizer(texts,
                     padding="max_length",
                     truncation=True,
                     max_length = 300,
                    )
print(t_inputs)

{'input_ids': [[2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 29871, 13, 26289, 310, 1404, 6030, 322, 5821, 2063, 29901, 13, 13, 1576, 1404, 11418, 952, 5192, 29893, 2817, 292, 29892, 1373, 4056, 29892, 322, 3165, 20657, 12298, 29892, 4049, 411, 263, 6017, 7716, 470, 3942, 29899, 12236, 287, 10929, 29889, 2688, 11188, 281, 986, 29891, 7928, 434, 29892, 3033, 6751, 5828, 9012, 29892, 322, 15319, 1319, 4890, 29889, 24980, 630, 12298, 322, 2351, 583, 393, 26987, 7333, 14321, 29892, 13852, 29892, 322, 5412, 3736, 1103, 3145, 373, 15839, 4959, 526, 884, 5025, 4395, 29889, 450, 1404, 29867, 304, 4772, 6501, 261, 29892, 901, 10676, 963, 267, 322, 758, 25534, 304, 1886, 261, 2821, 310, 4560, 29899, 7241, 322, 17268, 2531, 690, 29889, 13, 13, 1252, 9018, 362, 363, 2020, 278, 1404, 674, 763, 4104, 5570, 29901, 13, 13, 21878, 5570, 29892, 263, 1373, 4056, 322, 5192, 29893, 2817, 292, 3942, 2706, 29892, 

In [7]:
with torch.no_grad():
    last_hidden_state = model(**t_inputs, output_hidden_states = True)

AttributeError: 